In [1]:
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

In [2]:
pickle_file='notMNIST.pickle'
with open(pickle_file,'rb') as f:
    save=pickle.load(f)
    train_dataset=save['train_dataset']
    train_labels=save['train_labels']
    valid_dataset=save['valid_dataset']
    valid_labels=save['valid_labels']
    test_dataset=save['test_dataset']
    test_labels=save['test_labels']
    del save
    print('Training set',train_dataset.shape,train_labels.shape)
    print('Validation set',valid_dataset.shape,valid_labels.shape)
    print('Test set',test_dataset.shape,test_labels.shape)

Training set (20000, 28, 28) (20000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 0 to [1.0, 0.0, 0.0 ...], 1 to [0.0, 1.0, 0.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (20000, 784) (20000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [4]:
# With gradient descent training, even this much data is prohibitive.
# Subset the training data for faster turnaround.
train_subset = 10000

graph = tf.Graph()
with graph.as_default():

  # Input data.
  # Load the training, validation and test data into constants that are
  # attached to the graph.
  tf_train_dataset = tf.constant(train_dataset[:train_subset, :])
  tf_train_labels = tf.constant(train_labels[:train_subset])
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  # These are the parameters that we are going to be training. The weight
  # matrix will be initialized using random values following a (truncated)
  # normal distribution. The biases get initialized to zero.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  # We multiply the inputs with the weight matrix, and add biases. We compute
  # the softmax and cross-entropy (it's one operation in TensorFlow, because
  # it's very common, and it can be optimized). We take the average of this
  # cross-entropy across all training examples: that's our loss.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
  
  # Optimizer.
  # We are going to find the minimum of this loss using gradient descent.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  # These are not part of training, but merely here so that we can report
  # accuracy figures as we train.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See tf.nn.softmax_cross_entropy_with_logits_v2.



In [5]:
num_steps = 801

def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

with tf.Session(graph=graph) as session:
  # This is a one-time operation which ensures the parameters get initialized as
  # we described in the graph: random weights for the matrix, zeros for the
  # biases. 
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    # Run the computations. We tell .run() that we want to run the optimizer,
    # and get the loss value and the training predictions returned as numpy
    # arrays.
    _, l, predictions = session.run([optimizer, loss, train_prediction])
    if (step % 100 == 0):
      print('Loss at step %d: %f' % (step, l))
      print('Training accuracy: %.1f%%' % accuracy(
        predictions, train_labels[:train_subset, :]))
      # Calling .eval() on valid_prediction is basically like calling run(), but
      # just to get that one numpy array. Note that it recomputes all its graph
      # dependencies.
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Loss at step 0: 15.629375
Training accuracy: 12.0%
Validation accuracy: 15.2%
Loss at step 100: 2.411354
Training accuracy: 71.7%
Validation accuracy: 70.9%
Loss at step 200: 1.934071
Training accuracy: 74.8%
Validation accuracy: 73.2%
Loss at step 300: 1.670346
Training accuracy: 76.3%
Validation accuracy: 74.0%
Loss at step 400: 1.491545
Training accuracy: 77.3%
Validation accuracy: 74.3%
Loss at step 500: 1.358853
Training accuracy: 78.0%
Validation accuracy: 74.7%
Loss at step 600: 1.255640
Training accuracy: 78.4%
Validation accuracy: 74.7%
Loss at step 700: 1.172818
Training accuracy: 78.9%
Validation accuracy: 74.8%
Loss at step 800: 1.104622
Training accuracy: 79.4%
Validation accuracy: 74.9%
Test accuracy: 82.9%


In [6]:
batch_size = 128

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [7]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 18.843420
Minibatch accuracy: 10.2%
Validation accuracy: 12.3%
Minibatch loss at step 500: 1.191400
Minibatch accuracy: 75.8%
Validation accuracy: 73.8%
Minibatch loss at step 1000: 1.195883
Minibatch accuracy: 71.9%
Validation accuracy: 75.9%
Minibatch loss at step 1500: 1.206883
Minibatch accuracy: 77.3%
Validation accuracy: 75.8%
Minibatch loss at step 2000: 1.010649
Minibatch accuracy: 80.5%
Validation accuracy: 76.5%
Minibatch loss at step 2500: 0.691404
Minibatch accuracy: 82.0%
Validation accuracy: 76.6%
Minibatch loss at step 3000: 0.602773
Minibatch accuracy: 85.9%
Validation accuracy: 77.1%
Test accuracy: 84.5%


In [9]:
batch_size = 128  
num_hidden = 1024

graph = tf.Graph()  
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed  
  # at run time with a training minibatch.  
  tf_train_dataset = tf.placeholder(tf.float32,  
                                    shape=(batch_size, image_size * image_size))  
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))  
  tf_valid_dataset = tf.constant(valid_dataset)  
  tf_test_dataset = tf.constant(test_dataset)

  # Variables for linear layer 1  
  W1 = tf.Variable(  
    tf.truncated_normal([image_size * image_size, num_hidden]))  
  b1 = tf.Variable(tf.zeros([num_hidden]))

  # Hidden RELU input computation  
  y1 = tf.matmul(tf_train_dataset, W1) + b1  
  # Hidden RELU output computation  
  X1 = tf.nn.relu(y1)

  # Variables for linear layer 2  
  W2 = tf.Variable(  
    tf.truncated_normal([num_hidden, num_labels]))#W2  
  b2 = tf.Variable(tf.zeros([num_labels])) #b2  
  # logit (y2) output  
  logits = tf.matmul(X1, W2) + b2  
  loss = tf.reduce_mean(  
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))

  def getlogits(X):  
    y1 = tf.matmul(X, W1) + b1  
    X1 = tf.nn.relu(y1)  
    return tf.matmul(X1, W2) + b2

  # Optimizer.  
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

  # Predictions for the training, validation, and test data.  
  train_prediction = tf.nn.softmax(logits)  
  valid_prediction = tf.nn.softmax( getlogits(tf_valid_dataset) )  
  test_prediction = tf.nn.softmax( getlogits(tf_test_dataset))

ValueError: Only call `softmax_cross_entropy_with_logits` with named arguments (labels=..., logits=..., ...)